# Задание 1: лемматизация текста

## 1) Пути к файлам словаря и текста, шаблоны парсера

In [1]:
dictInputPath = '../dict.opcorpora.txt'
textInputPath = './sampletext.txt'

In [2]:
wordTemplate = r'(([А-Яа-яЁё][a-яё]*)+(-[А-Яа-яЁё][а-яё]*)*)+'
groupNumberTemplate = r'^[\d]+$'

## 2) Импорт библиотек

In [3]:
import re

## 3) Объявление классов: теги, словарь; инициализация словаря

In [4]:
class Tags:
    def __init__(self, primaryForm: str, partOfSpeech: str):
        self.primaryForm = primaryForm
        self.partOfSpeech = partOfSpeech
    
    def __str__(self):
        return f'[{self.primaryForm}={self.partOfSpeech}]'
    
    def __eq__(self, other):
        if self.primaryForm != other.primaryForm:
            return False
        return self.partOfSpeech == other.partOfSpeech

<b>Замечание:</b> Словарь приравнивает букву "ё" к "е", но во избежание коллизий хранит для омографов списки тегов (обходной путь для случая бесконтекстной лемматизации).

In [5]:
class WordTagsDictionary:
    def __init__(self, txtPath: str = None):
        self.dictionary = dict()
        self.txtPath = txtPath
        if(self.txtPath != None):
            self.parseTxt()
    
    def _enterWord(self, word: str, primaryForm: str, tags):
        if word.find('Ё'):
            word = word.replace('Ё', 'Е')
        if not (word in self.dictionary.keys()):
            self.dictionary[word] = []
        
        newTags = Tags(primaryForm, tags[0])
        doAddTags = True
        
        for tag in self.dictionary[word]:
            if newTags == tag:
                doAddTags = False
                break
        
        if doAddTags:
            self.dictionary[word].append(newTags)
    
    
    def parseTxt(self):
        with open(self.txtPath, 'r', encoding='utf-8') as file:
            primaryForm = ''
            wordGroup = 0
            newWordGroup = 0
            
            for line in file:
                if line == '\n':
                    continue
                if re.search(groupNumberTemplate, line):
                    newWordGroup = int(line)
                else:
                    word, tagsJoined = line.split('\t')
                    tags = re.split(',| |\n', tagsJoined)
                    if wordGroup != newWordGroup:
                        wordGroup = newWordGroup
                        primaryForm = word
                    self._enterWord(word, primaryForm, tags)
                    
            file.close()
    
    def getTags(self, word: str):
        key = word.upper().replace('Ё', 'Е')
        if(key in self.dictionary.keys()):
            return self.dictionary[key]
        return None

In [6]:
dictionary = WordTagsDictionary(dictInputPath)

## 4) Предобработка входных данных

In [7]:
def textToWords(text: str):
    wordList = []
    result = re.findall(wordTemplate, text)
    for word in result:
        wordList.append(word[0])
    return wordList

## 5) Лемматизация и разметка тегами

In [8]:
with open(textInputPath, 'r', encoding='utf-8') as file:
    for line in file:
        words = textToWords(line)
        print(line, end='\n\n')
        for word in words:
            print(word, end='')
            for tag in dictionary.getTags(word):
                print(tag, end='')
            print('', end=' ')
        print('\n')

В туннеле становилось всё холоднее: мы с Мельником поднимались всё ближе и ближе к поверхности. Через минуту мы должны были оказаться наверху, наедине с воющим ветром, наедине с чудовищами, наедине с самой судьбой… я был почти у цели.


В[В=PREP][В=NOUN] туннеле[ТУННЕЛЬ=NOUN] становилось[СТАНОВЛЮСЬ=VERB] всё[ВЕСЬ=ADJF][ВСЁ=PRCL] холоднее[ХОЛОДНЕЕ=COMP] мы[МЫ=NPRO] с[С=PREP][С=NOUN][С=PRCL] Мельником[МЕЛЬНИК=NOUN] поднимались[ПОДНИМАЮСЬ=VERB] всё[ВЕСЬ=ADJF][ВСЁ=PRCL] ближе[БЛИЖЕ=COMP] и[И=CONJ][И=INTJ][И=PRCL][И=NOUN] ближе[БЛИЖЕ=COMP] к[К=PREP][КАК=CONJ][К=NOUN] поверхности[ПОВЕРХНОСТЬ=NOUN] Через[ЧЕРЕЗ=PREP] минуту[МИНУТА=NOUN] мы[МЫ=NPRO] должны[ДОЛЖЕН=ADJS] были[БЫЛЬ=NOUN][ЕСТЬ=VERB] оказаться[ОКАЗАТЬСЯ=INFN] наверху[НАВЕРХУ=ADVB] наедине[НАЕДИНЕ=ADVB] с[С=PREP][С=NOUN][С=PRCL] воющим[ВОЮЩИЙ=PRTF] ветром[ВЕТЕР=NOUN] наедине[НАЕДИНЕ=ADVB] с[С=PREP][С=NOUN][С=PRCL] чудовищами[ЧУДОВИЩЕ=NOUN] наедине[НАЕДИНЕ=ADVB] с[С=PREP][С=NOUN][С=PRCL] самой[САМ=ADJF][САМЫЙ=ADJF] судьбой[СУДЬБА=NOUN